In [1]:
import time
from multiprocessing import Pool
import functools

In [2]:
d="/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction/"
dd="/media/some/sophie-hhd/transmitt/fic/Fictional Literature/"
files=! find "$d" "$dd" -type f -iname "*.epub"
len(files)

8837

In [ ]:
#test
f=files[-4053]
#print f
getFileIsbnMeta(f,.2)

In [11]:
titles=load_obj('data/titlesISBN')#{}
keys=load_obj('data/titleskeysISBN')#[]

In [16]:
flist=list(filter(lambda x: x[x.rindex('/')+1:x.rindex('.')] not in keys, files))
len(flist),len(keys),len(files)

(8834, 8, 8837)

In [17]:

t0 = time.time()
books=[]
cnt=0

#books=list(map(getFileIsbnMeta, flist[-10:]))
for f in flist[10:100]:
    books.append(getFileIsbnMeta(f))
    time.sleep(10) 
#print "errror" 
##p.terminate()
#p.close()
#p.terminate()
#p.close()
for book in books:
    try:
        if book[0] not in keys and len(book)>2:
            titles[book[0]]=book[1:]
    except:
        cnt+=1
print "exception count ",cnt        
print len(list(titles.keys()))
save_obj(titles, 'data/titlesISBN')
keys=list(titles.keys())
save_obj(keys,'data/titleskeysISBN')
print 'finish parallel'

t1=time.time()
total = t1-t0
print('time: ',total)

exception count  0
52
finish parallel
('time: ', 2066.2928919792175)


In [4]:
f='/media/some/sophie-hhd/transmitt/fic/Fictional Literature/True Crime Collection/Portrait of a Killer_ Jack the Ripper Case Closed - Patricia Cornwell/Patricia Cornwell - Portrait of a Killer_ Jack the_sed (v5.0).epub'

In [8]:
getFileIsbnMeta(f)

'NO meta'

In [14]:
import os
import isbnlib
import string
import comFuncs
from comFuncs import *
from isbnlib.config import add_apikey

import os
import string

print "api initialised"
def getISBNmeta(title,author):
    APIKEY="2E7FE5A4"
    SERVICE = 'isbndb'
    #register your key
    add_apikey(SERVICE, APIKEY)
    #print(author, title)
    #isbn=isbnlib.isbn_from_words(title+" by " +author)
    if author.count(' ')>0:
        author=' '.join(author.split())
    if title.count(' ')>0:
        title=' '.join(title.split())  
    cnt=0
    while cnt<5:
        try:
            m=isbnlib.goom(title+" " +author)
            return m
        except:
                cnt+=1
                time.sleep(10)
                pass
        return ""
        #print "service unavailable"
def getFileIsbnMeta(f,smscr=1.0):
    #print "start"
    f=f.encode('ascii','ignore')
    fp,ff= os.path.split(f)
    fn, fext = os.path.splitext(ff)
    fn=re.sub(" \(v[0-9].[0-9]\)","",fn)
    #print fn
    seri=''
    sm=0
    mtd=None
    sp=fn.split()
    if len(sp)>4:
        tit=sp[-2:]
        auth=sp[:-2]
    elif len(sp) >1:
        tit=fn.split()[-1:]
        auth=fn.split()[:-1]
    else:
        return "no meta"
    
    if fn.count(' - ')==1:
        tit,auth=fn.split(' - ')
    elif fn.count(' - ')==2:
        seri,tit,auth=fn.split(' - ')
        
    #print "title",tit,"author",auth
   
    if tit.count('_ ')>0:
        spp=tit.split('_')[0]
        #print spp
        if spp.count(' ')>0:
            tit=' '.join(spp.split())
        else:
            tit=spp
    if auth.count('_ ')>0:
        spp=auth.split('_')[0]
        #print spp
        if spp.count(' ')>0:
            auth=' '.join(spp.split())
        else:
            auth=spp
    if tit.count(' ')>0:
        title=''.join(l for l in ' '.join(tit.split()) if l in string.printable)
    else:
        title=''.join(l for l in tit if l in string.printable)
    if auth.count(" ")>0:    
        author=''.join(l for l in ' '.join(auth.split()) if l in string.printable)
    else:
        author=''.join(l for l in auth if l in string.ascii_letters+" .")
    #print tit,auth
    #print title,'..... ....',author
  
    metad =getISBNmeta(title,author)
    if metad !="" and type(metad)==list: 
        for md in metad:
            if type(md)==dict and "Authors" in md.keys() and "Title" in md.keys():
                if len(md['Authors'])>0:
                    authsim=similarity(md['Authors'][0].encode('ascii','ignore'),str(author))
                else:
                    authsim=0
                titsim=similarity(md['Title'].encode('ascii','ignore'),str(title))
                #print titsim,authsim
                if titsim+authsim >sm:
                    sm=titsim+authsim
                    mtd=md
    if sm >smscr and mtd !=None:
        return fn,fp,mtd,sm
    #if no data found switch title and author
    metad =getISBNmeta(author,str(title))
    if metad !="" and type(metad)==list: 
        for md in metad:
            if type(md)==dict and "Authors" in md.keys() and "Title" in md.keys():
                if len(md['Authors'])>0:
                    authsim=similarity(md['Authors'][0].encode('ascii','ignore'),str(title))
                else:
                    authsim=0
                titsim=similarity(md['Title'].encode('ascii','ignore'),str(author))
                #print titsim,authsim
                if titsim+authsim >sm:
                    sm=titsim+authsim
                    mtd=md
    if sm >smscr and mtd !=None:
        return fn,fp,mtd,sm
    # if no data found try seri, auth ,title  combinaton
    if seri !='':
        metad =getISBNmeta(title,str(seri))
        if metad !="" and type(metad)==list:
            
            for md in metad:
                if type(md)==dict and "Authors" in md.keys() and "Title" in md.keys():
                    if len(md['Authors'])>0:
                        authsim=similarity(md['Authors'][0].encode('ascii','ignore'),str(seri))
                    else:
                        authsim=0
                    titsim=similarity(md['Title'].encode('ascii','ignore'),str(title))
                    #print titsim,authsim
                    if titsim+authsim >sm:
                        sm=titsim+authsim
                        mtd=md
        if sm >smscr and mtd !=None:
            return fn,fp,mtd,sm
        metad =getISBNmeta(seri,str(author))
        if metad !="" and type(metad)==list: 
            for md in metad:
                if type(md)==dict and "Authors" in md.keys() and "Title" in md.keys():
                    if len(md['Authors'])>0:
                        authsim=similarity(md['Authors'][0].encode('ascii','ignore'),str(author))
                    else:
                        authsim=0
                    titsim=similarity(md['Title'].encode('ascii','ignore'),str(seri))
                    #print titsim,authsim
                    if titsim+authsim >sm:
                        sm=titsim+authsim
                        mtd=md    
        if sm >smscr and mtd !=None:
            return fn,fp,mtd,sm
        metad =getISBNmeta(seri,str(title))
        if metad !="" and type(metad)==list: 
            for md in metad:
                if type(md)==dict and "Authors" in md.keys() and "Title" in md.keys():
                    if len(md['Authors'])>0:
                        authsim=similarity(md['Authors'][0].encode('ascii','ignore'),str(title))
                    else:
                        authsim=0
                    titsim=similarity(md['Title'].encode('ascii','ignore'),str(seri))
                    #print titsim,authsim
                    if titsim+authsim >sm:
                        sm=titsim+authsim
                        mtd=md
        if sm >smscr and mtd !=None:
            return fn,fp,mtd,sm
        metad =getISBNmeta(author,str(seri))
        if metad !="" and type(metad)==list: 
            for md in metad:
                if type(md)==dict and "Authors" in md.keys() and "Title" in md.keys():
                    if len(md['Authors'])>0:
                        authsim=similarity(md['Authors'][0].encode('ascii','ignore'),str(seri))
                    else:
                        authsim=0
                    titsim=similarity(md['Title'].encode('ascii','ignore'),str(author))
                    #print titsim,authsim
                    if titsim+authsim >sm:
                        sm=titsim+authsim
                        mtd=md               
    if sm >smscr and mtd !=None:
        return fn,fp,mtd,sm
    return "NO meta"   
    

api initialised


In [18]:
query="The+Moving+Finger+Agatha+Christie"

In [ ]:
urlstring='https://www.googleapis.com/books/v1/volumes?q='+query+'&fields=items/volumeInfo(title,authors)&maxResults=10'